## 为什么要定义消息格式
消息头存放消息的格式数据， 比如 消息的长度、类型、状态等等， 而消息体存放具体的传送数据。

对于使用TCP协议传输信息的程序来说，格式定义一定要明确规定 消息的边界 。

因为 TCP协议传输的是 字节流（bytes stream）， 如果消息中没有指定 边界 或者 长度，接收方就不知道一个完整的消息从字节流的 哪里开始，到 哪里结束。


指定消息的边界有两种方式：

* 用特殊字节作为消息的结尾符号

可以用消息内容中不可能出现的字节串 （比如 FFFFFF） 作为消息的结尾字符。

* 在消息开头某个位置，直接指定消息的长度

比如在一个消息的最前面用2个字节表示本消息的长度。


>UDP协议通常不需要指定消息边界，因为UDP是数据报协议，应用程序从socket接收到的必定是发送方发送的完整消息。

## 示例1

我们现在要开发一个实验室的工作站监控系统，包括

* 安装在机房工作站上的 数据采集器 RUS
  这个程序作为TCP服务端，获取资源使用数据，简称 RUS （Resource Usage Stat）

* 安装在监控室的管理控制台 AT
  这个程序作为TCP客户端，向管理员显示资源使用数据，简称 AT （Admin Terminal）

下面是一种参考的规范：

* AT 和 RUS 之间采用 TCP 长连接方式进行通讯，如果中途出现连接断开，AT 作为 TCP 客户端必须进行重连

* 消息整体

    * 每个消息 都是 UTF8 编码的 字符串，由消息头 和消息体组成。

    * 消息头 和消息体之间 用一个 换行符 （UTF8编码后的字节为 0A ）隔开。

有如下类型的消息：

控制命令

由 AT 发送给 RUS ， 下达管理控制命令。

比如：

* pause 暂停数据采集
* resume 恢复数据采集
RUS接收到 控制命令后，必须完成操作后必须回复响应消息，告诉 AT 命令已经接收已经完成

数据上报

* 由 RUS 发送给 AT，汇报采集的资源数据。 AT接收到数据后，应该回复一个接收汇报的响应消息。

消息头

* 消息头只包含一个信息： 消息体的长度

* 消息头用十进制的字符串 表示一个整数的长度

消息体

* 消息体用json格式的字符串 表示数据信息，如下

* 数据上报 RUS -> AT
```json
{
    "type" : "report",
    "info" : {
        "CPU Usage" : "30%",
        "Mem usage" : "53%"
    }
}
```
* 数据上报响应 AT -> RUS
```json
{
    "type" : "report-ack"
}
```
* 暂停数据上报命令 AT -> RUS
```json
{
    "type" : "pause",
    "duration" :  200
}
```
其中 duration 表示暂停上报的时间，以秒为单位

* 恢复数据上报命令 AT -> RUS
```json
{
    "type" : "resume"
}
```
* 命令处理响应 RUS -> AT
```json
{
    "type" : "cmd-ack",
    "code" :  200,
    "info" : "处理成功"
}
```
其中code 是处理结果码，用200表示成功。 info 是处理结果文字描述。

### RUS：

### AT：

## 示例2:

数据都用字符表示，是比较浪费带宽的做法。

比如:返回码 用 200 这样的字符串表示，就会耗费3个字节，24个比特。 如果处理结果 只有成功和不成功，只需要1个bit 即可， 1表示成功，0表示不成功


其次， json这种复杂语法的编解码算法，需要程序代码进行各种复杂处理（参考一下Python json内置库的代码）是比较耗费CPU 资源的。

可以定义更为简单的数据表达方式，比如像这样：

消息头 开头2个字节表示消息的长度

消息头 第3个字节表示 消息类型 ：

0：暂停命令， 1 ：恢复命令 2：命令响应 3：统计上报 4：统计上报响应

消息体 数据定义

可以使用类似 Radius/Diameter Attribute-Value Pairs （AVP） 的定义方法

Attribute： 使用一个字节，表示数据种类。

比如

1： CPU 使用率 2： 内存使用率

Length： 使用一个字节，表示信息长度

Value： 表示具体的数据值

这样，前面的示例信息
```json

 {
        "CPU Usage" : "30%",
        "Mem usage" : "53%"
 }
```  
其中:

* "CPU Usage" : "30%" , 就像这样用16进制字节表示 01011E

* "Mem usage" : "53%" , 就像这样用16进制字节表示 020135

合起来就是 **01011E020135**

对比一下，第一种编码方法

* 优点：更节省传输带宽，编码解码数据效率更高

* 缺点：对于人的可读性差，数据表示灵活性较差；